In [2]:
import polars as pl
import datetime
import holidays

def is_weekend(date: datetime):
    return date.weekday() > 5

def is_holiday(date: datetime):
    return date.date() in holidays.BE()


In [3]:
# we want to figure out which non-weekend or holiday days attracted the most visitors.
(
    pl
        .read_parquet("../data/visitors.parquet")
        .filter(pl.col("timestamp").map_elements(lambda dt: !is_weekend(dt) and !is_holiday(dt)))

)

timestamp,visitors
datetime[μs],i16
2020-04-20 18:00:00,2328
2020-04-21 18:00:00,2028
2020-04-22 18:00:00,2268
2020-04-23 18:00:00,2128
2020-04-24 18:00:00,2026
…,…
2024-04-13 18:00:00,2730
2024-04-14 18:00:00,2603
2024-04-15 18:00:00,2244
